In [21]:
import pandas as pd

# Excelファイルのパス
file_path = "第67回東医体スキー競技スタートリスト_大学別.xlsx"

# すべてのシートを対象に、大学ごとのBib番号・Bib枚数を集計する
xls = pd.ExcelFile(file_path)
bib_summary_all = {}

for sheet in xls.sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet)

    # bib列のデータ型を確認
    df["bib"] = pd.to_numeric(df["bib"], errors="coerce")  # 数値変換し、変換できないものはNaNにする
    df = df.dropna(subset=["bib"])  # NaNを削除
    df["bib"] = df["bib"].astype(int)  # 整数型に変換
    
    # "大学" 列ごとにグループ化し、"bib" 列のリストを作成
    grouped = df.groupby("大学")["bib"].apply(lambda x: ', '.join(map(str, x))).reset_index()
    
    # Bibの枚数を追加
    grouped["Bib枚数"] = grouped["bib"].apply(lambda x: len(x.split(', ')))
    
    # 結果を保存
    bib_summary_all[sheet] = grouped

# 結果を新しいExcelファイルに保存
output_file = "Bib集計結果.xlsx"
with pd.ExcelWriter(output_file) as writer:
    for sheet, df in bib_summary_all.items():
        df.to_excel(writer, sheet_name=sheet, index=False)

print(f"結果を {output_file} に保存しました！")

結果を Bib集計結果.xlsx に保存しました！


In [22]:
import openpyxl

# 大学名と対応番号の辞書
university_dict = {
    "北海道大学": 1, "弘前大学": 2, "東北大学": 3, "秋田大学": 4, "山形大学": 5,
    "筑波大学": 6, "群馬大学": 7, "千葉大学": 8, "東京大学": 9, "東京科学大学": 10,
    "新潟大学": 11, "山梨大学": 12, "信州大学": 13, "旭川医科大学": 14, "札幌医科大学": 15,
    "福島県立医科大学": 16, "岩手医科大学": 17, "東北医科薬科大学": 18, "自治医科大学": 19, "獨協医科大学": 20,
    "埼玉医科大学": 21, "国際医療福祉大学": 22, "杏林大学": 23, "慶応義塾大学": 24, "順天堂大学": 25,
    "昭和大学": 26, "帝京大学": 27, "東京医科大学": 28, "東京慈恵会医科大学": 29, "東京女子医科大学": 30,
    "東邦大学": 31, "日本大学": 32, "日本医科大学": 33, "北里大学": 34, "聖マリアンナ医科大学": 35, "東海大学": 36
}

# Excelファイルを読み込み
wb = openpyxl.load_workbook("Bib集計結果.xlsx")

# すべてのシートを処理
for sheet_name in wb.sheetnames:
    ws = wb[sheet_name]  # 現在のシートを取得
    
    # 列を左に追加（A列に大学番号を追加）
    ws.insert_cols(1)
    
    # ヘッダー追加
    ws.cell(1, 1).value = "大学番号"
    
    # D列（4列目）の大学名をチェックし、A列（1列目）に番号を入力
    for row in range(2, ws.max_row + 1):  # 2行目から最終行まで処理
        university_name = ws.cell(row, 2).value  # もともとD列（4列目）だった列が、1列追加でE列（5列目）になる
        
        if university_name in university_dict:
            ws.cell(row, 1).value = university_dict[university_name]  # A列に対応する番号を入力
        else:
            ws.cell(row, 1).value = ""  # 空白にする
    
    # ソートのためにデータを取得
    data = []
    for row in range(2, ws.max_row + 1):
        num = ws.cell(row, 1).value  # 大学番号
        row_values = [ws.cell(row, col).value for col in range(1, ws.max_column + 1)]  # すべての列を取得
        if num is not None:
            data.append(row_values)

    # 大学番号で昇順ソート
    data.sort(key=lambda x: x[0])

    # 並び替えたデータを書き戻す
    for row_idx, row_data in enumerate(data, start=2):
        for col_idx, value in enumerate(row_data, start=1):
            ws.cell(row_idx, col_idx, value)

# 保存
wb.save("Bib集計結果_並び替え.xlsx")
print("すべてのシートで大学番号を追加し、昇順に並び替えました！")

すべてのシートで大学番号を追加し、昇順に並び替えました！
